In [90]:
import pandas as pd
from IPython.display import display

In [91]:
rep="/Users/gilles/Downloads/"
fLexFilms4=rep+"Lexique4_min+filtr+cat+separ+corr3.tsv"
df=pd.read_csv(filepath_or_buffer=fLexFilms4,sep="\t",header=None,skip_blank_lines=True,encoding="utf8")
df.columns="graphie lemme cat infover genre nombre occurrences".split(" ")
df.head()

,graphie,lemme,cat,infover,genre,nombre,occurrences
0,fossiles,fossile,ADJ,NaN,NaN,p,41.0
1,clopes,cloper,VER,sub:pre:2,NaN,s,1.0
2,ardeurs,ardeur,NOM,NaN,f,p,143.0
3,translations,translation,NOM,NaN,NaN,NaN,3.0
4,antidrogue,antidrogue,ADJ,NaN,m,s,36.0


In [92]:
def bilanDF(lDF,title="cas"):
    print "%s : %d"%(title,len(lDF))
    display(lDF.head())
    return 

# Contrôles de cohérence

In [93]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pdMaxRows=pd.options.display.max_rows
pd.set_option('display.max_rows', 30)

## Genre & Nombre
Pour les adjectifs et les noms, on cherche toutes les formes qui ont 
- un genre explicite en dehors de m, f 
- un nombre explicite en dehors de s, p

### Genre

In [94]:
condGenre=(df["cat"].isin(["NOM","ADJ"]))&(df["genre"].notnull())&(~df["genre"].isin(["f","m"]))
bilanDF(df[condGenre],"Substantifs hors m/f")

Substantifs hors m/f : 2


,graphie,lemme,cat,infover,genre,nombre,occurrences
96511,gluant,gluant,ADJ,par:p,e,r,24.0
157666,attrayant,attrayant,ADJ,par:p,e,r,16.0


### Nombre

In [95]:
condNombre=(df["cat"].isin(["NOM","ADJ"]))&(df["nombre"].notnull())&(~df["nombre"].isin(["s","p"]))
bilanDF(df[condNombre],"Substantifs hors s/p")

Substantifs hors s/p : 2


,graphie,lemme,cat,infover,genre,nombre,occurrences
96511,gluant,gluant,ADJ,par:p,e,r,24.0
157666,attrayant,attrayant,ADJ,par:p,e,r,16.0


Il y a seulement un problème avec *gluant* et *attrayant*, qui ont été classés comme ADJ, parce que l'étiquette par:pre a été mal traitée. Il y a un problème plus général avec les participes présents qui ont aussi un infover par:p avec le genre e et le nombre r, probablement suite à la même erreur.

#### Pluriels non marqués par s/x/z

In [96]:
condNomPl=(df["cat"].isin(["NOM"]))&(~df["graphie"].str[-1:].isin([u"s",u"x",u"z"]))&(df["nombre"].isin(["p"]))
bilanDF(df[condNomPl],"Noms p")

Noms p : 189


,graphie,lemme,cat,infover,genre,nombre,occurrences
419,condottieri,condottieri,NOM,NaN,m,p,4.0
4754,oeils-de-chat,oeils-de-chat,NOM,NaN,m,p,1.0
5510,missi dominici,missi dominici,NOM,NaN,m,p,1.0
6453,laissées-pour-compte,laissées-pour-compte,NOM,NaN,f,p,3.0
6733,pots-de-vin,pots-de-vin,NOM,NaN,m,p,589.0


## Infover

### participes présent
- l'étiquetage des participes présent n'est pas conforme à la nomenclature par:pre (3634 formes)
 - le découpage a donné infover=par:p et le re s'est retrouvé découpé entre nombre et genre

In [97]:
condParPre=(df["infover"]=="par:p")
bilanDF(df[condParPre],"infover=par:p")

infover=par:p : 3634


,graphie,lemme,cat,infover,genre,nombre,occurrences
17,rivalisant,rivaliser,VER,par:p,e,r,22.0
64,surprotégeant,surprotéger,VER,par:p,e,r,1.0
183,migrant,migrer,VER,par:p,e,r,4.0
189,instruisant,instruire,VER,par:p,e,r,7.0
257,fredonnant,fredonner,VER,par:p,e,r,52.0


### Infover sans verbe
- infover ne devait pas être présent en dehors des formes verbales
 - pour les adjectifs, il faudrait sans doute prendre une décision pour ceux qui ressemblent à des participes passés
 - pour les noms, il faudrait probablement corriger


In [98]:
condInfover=(~df["cat"].isin(["VER","AUX"]))&(df["infover"].notnull())
bilanDF(df[condInfover],"infover hors formes verbales")

infover hors formes verbales : 228


,graphie,lemme,cat,infover,genre,nombre,occurrences
538,cymbale,cymbale,NOM,ind:pre:1,NaN,s,2.0
1308,mitre,mitre,NOM,ind:pre:3,NaN,s,1.0
2360,camarades,camarade,NOM,sub:pre:2,NaN,s,19.0
3647,skipper,skipper,NOM,inf,NaN,NaN,12.0
5127,portraits,portrait,NOM,par:pas:,m,p,11.0


### Verbe sans infover
Toutes les formes verbales devraient avoir un infover mais 2328 formes n'ont aucune information associée

In [99]:
condVerInfover=(df["cat"].isin(["VER","AUX"]))&(df["infover"].isnull())
bilanDF(df[condVerInfover],"formes verbales sans infover")

formes verbales sans infover : 2328


,graphie,lemme,cat,infover,genre,nombre,occurrences
26,voulez,vouloir,VER,NaN,NaN,NaN,9.0
130,renifles,renifler,VER,NaN,NaN,NaN,1.0
141,donnera,donner,VER,NaN,NaN,NaN,2.0
368,tuerai,tuer,VER,NaN,NaN,NaN,1.0
421,alerter,alerter,VER,NaN,NaN,NaN,1.0


### Cohérence des formes graphiques
Tester la plausibilité de infover en fonction de la graphie
- "er" n'est compatible qu'avec inf

**⚠️ Problème avec la comparaison de chaine utf8, la condition d'égalité donne un message d'erreur à propos de l'encodage.**
```
condPP=(df["cat"].isin(["VER","AUX"]))&(df["graphie"].str[-1:]==u"é")
```
**La comparaison avec une liste fonctionne...**

### Infinitif
- la forme en -er doit être associée à inf

In [100]:
condInf=((df["cat"].isin(["VER","AUX"]))
         &(df["graphie"].str[-2:]=="er")
         &(~df["infover"].isin(["inf","inf;"]))&df["infover"].notnull())
bilanDF(df[condInf],"infinitifs hors inf")

infinitifs hors inf : 1201


,graphie,lemme,cat,infover,genre,nombre,occurrences
134,empoisonner,empoisonner,VER,ind:pre:2,NaN,p,1.0
262,monter,monter,VER,imp:pre:2,NaN,p,1.0
407,dispenser,dispenser,VER,ind:pre:2,NaN,p,1.0
627,abuser,abuser,VER,ind:pre:2,NaN,p,1.0
666,nettoyer,nettoyer,VER,imp:pre:2,NaN,p,1.0


Les 1201 formes en -er précédentes sont classés comme des formes finies.

### Participes passé
- les formes en -é.e.s doivent être associées à par:pas:
 - sauf exception pour les verbes en -éer

In [101]:
condNotPP=((df["cat"].isin(["VER","AUX"]))
           &(~df["infover"].isin(["par:pas:"])
           &(df["infover"].notnull())))

condFinPP=((df["graphie"].str[-1:].isin([u"é"]))
           |(df["graphie"].str[-2:].isin([u"ée",u"és"]))
           |(df["graphie"].str[-3:].isin([u"ées"])))
condLemmeEER=(df["lemme"].str[-3:].isin([u"éer"]))
display(bilanDF(df[condNotPP&condFinPP&~condLemmeEER],"formes en é.e.s hors participes passé, hors verbes en -éer"))
bilanDF(df[condNotPP&condFinPP&condLemmeEER],"formes en é.e.s hors participes passé pour les verbes en -éer")

formes en é.e.s hors participes passé, hors verbes en -éer : 1163


,graphie,lemme,cat,infover,genre,nombre,occurrences
90,nommé,nommer,VER,ind:imp:3,NaN,s,1.0
319,effrayée,effrayer,VER,ind:imp:3,NaN,s,1.0
445,nommé,nommer,VER,ind:imp:3,NaN,p,4.0
494,utilisées,utiliser,VER,ind:imp:3,NaN,s,1.0
659,défoncé,défoncer,VER,ind:imp:3,NaN,p,1.0


None

formes en é.e.s hors participes passé pour les verbes en -éer : 48


,graphie,lemme,cat,infover,genre,nombre,occurrences
2695,énuclée,énucléer,VER,ind:pre:1,NaN,s,1.0
5163,recrée,recréer,VER,imp:pre:2,NaN,s,2.0
20384,crée,créer,VER,ind:pre:1,NaN,s,469.0
23874,guée,guéer,VER,ind:pre:3,NaN,s,4.0
33917,crée,créer,VER,imp:pre:2,NaN,s,239.0


Sur les 48 formes repérées pour les verbes en -éer, certaines peuvent être incorrectes.

### 2PL et -ez
Pour les formes 2PL, la finale doit être -ez sauf pour le passé simple
- sauf êtes, dites, faites (non traitées)

In [102]:
cond2P=((df["cat"].isin(["VER","AUX"]))
        &(df["infover"].str.contains("2"))
        &(~df["infover"].str.contains("ind:pas", na=False))
        &(df["nombre"]=="p"))

condFin2P=(~df["graphie"].str[-2:].isin([u"ez"]))
bilanDF(df[cond2P&condFin2P],u"2PL hors -ez")

2PL hors -ez : 1323


,graphie,lemme,cat,infover,genre,nombre,occurrences
134,empoisonner,empoisonner,VER,ind:pre:2,NaN,p,1.0
262,monter,monter,VER,imp:pre:2,NaN,p,1.0
407,dispenser,dispenser,VER,ind:pre:2,NaN,p,1.0
627,abuser,abuser,VER,ind:pre:2,NaN,p,1.0
666,nettoyer,nettoyer,VER,imp:pre:2,NaN,p,1.0


### 1PL et -ons
Pour les formes 1PL, la finale doit être -ons sauf pour le passé simple
 - exception sommes (non traitée)

In [103]:
cond1P=((df["cat"].isin(["VER","AUX"]))
        &(df["infover"].str.contains("1"))
        &(~df["infover"].str.contains("ind:pas", na=False))
        &(df["nombre"]=="p"))

condFin1P=(~df["graphie"].str[-3:].isin([u"ons"]))
bilanDF(df[cond1P&condFin1P],"1PL hors -ons")

1PL hors -ons : 61


,graphie,lemme,cat,infover,genre,nombre,occurrences
566,accordes,accorder,VER,ind:pre:1,NaN,p,1.0
1776,couvres,couvrir,VER,ind:pre:1,NaN,p,1.0
3404,laisses,laisser,VER,ind:pre:1,NaN,p,16.0
6524,indiques,indiquer,VER,ind:pre:1,NaN,p,1.0
10491,trouves,trouver,VER,ind:pre:1,NaN,p,1.0
